# Outlines Pydantic Program

Generate structured data with [**outlines**](https://github.com/normal-computing/outlines) via LlamaIndex.  

(Taken from the repo README): Outlines 〰 helps developers guide text generation to build robust interfaces with external systems. Provides generation methods that guarantee that the output will match a regular expressions, or follow a JSON schema.

In this example we show how we can use `outlines` to extract structured pydantic objects from unstructured text.

In [ ]:
# install `outlines` if you haven't done so already
!pip install outlines

In [10]:
from pydantic import BaseModel
from typing import List
from enum import Enum

from llama_index.program import OutlinesProgram

Define output schema

In [11]:
class Weapon(str, Enum):
    sword = "sword"
    axe = "axe"
    mace = "mace"
    spear = "spear"
    bow = "bow"
    crossbow = "crossbow"


class Armor(str, Enum):
    leather = "leather"
    chainmail = "chainmail"
    plate = "plate"


class Character(BaseModel):
    name: str
    age: int
    armor: Armor
    weapon: Weapon
    strength: int

Define outlines pydantic program

In [12]:
import outlines.models as models

program = OutlinesProgram.from_defaults(
    output_cls=Character,
    prompt_template_str="{input_str}",
    llm=models.OpenAICompletion("gpt-3.5-turbo", max_tokens=256, temperature=0.1),
)

Run program to get structured output.  
Text highlighted in blue is variables specified by us, text highlighted in green is generated by the LLM.

In [13]:
Character.model_json_schema()

{'title': 'Character',
 'type': 'object',
 'properties': {'name': {'title': 'Name', 'type': 'string'},
  'age': {'title': 'Age', 'type': 'integer'},
  'armor': {'$ref': '#/$defs/Armor'},
  'weapon': {'$ref': '#/$defs/Weapon'},
  'strength': {'title': 'Strength', 'type': 'integer'}},
 'required': ['name', 'age', 'armor', 'weapon', 'strength'],
 '$defs': {'Armor': {'title': 'Armor',
   'description': 'An enumeration.',
   'enum': ['leather', 'chainmail', 'plate'],
   'type': 'string'},
  'Weapon': {'title': 'Weapon',
   'description': 'An enumeration.',
   'enum': ['sword', 'axe', 'mace', 'spear', 'bow', 'crossbow'],
   'type': 'string'}}}

In [14]:
import outlines.models as models
import outlines.text.generate as generate

# model = models.transformers("gpt2-medium")
model = models.OpenAICompletion("gpt-3.5-turbo", max_tokens=256, temperature=0.1)

sequence = generate.json(model, Character)("Give me a character description")

AttributeError: 'function' object has no attribute 'device'

In [ ]:
output = program(input_str="Generate a random character description")

The output is a valid Pydantic object that we can then use to call functions/APIs. 

In [ ]:
output